<a href="https://colab.research.google.com/github/srdg/ISL-T/blob/main/src/ISL_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake because of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


In [ ]:
from IPython.display import YouTubeVideo
YOUTUBE_ID = 'gFyDUuy1_88'
YouTubeVideo(YOUTUBE_ID)

In [ ]:
# install youtube-dl
!pip install youtube-dl
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" \
https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 10 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 10 video.mp4
# detect poses on the these 10 seconds
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 \
--write_json ./output/ --display 0  --write_video ../openpose.avi -- face --hands
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=560, height=340)

In [ ]:
# cut the first 10 seconds
!ffmpeg -y -loglevel info -i loud/MVI_5177.MOV -t 10 video_loud.mp4
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video_loud.mp4 \
--write_json ./output/ --display 0  --write_video ../openpose.avi -- face --hands
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output_loud.mp4

In [ ]:
show_local_mp4_video('output_loud.mp4', width=560, height=340)

In [ ]:
# cut the first 10 seconds
!ffmpeg -y -loglevel info -i loud/MVI_5178.MOV video_loud_2.mp4
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video_loud_2.mp4 \
--write_json ./output/ --display 0  --write_video ../openpose.avi -- face --hands
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output_loud_2.mp4

In [ ]:
show_local_mp4_video('output_loud_2.mp4', width=560, height=340)

In [ ]:
!pip install youtube-dl

!wget -O actor_1.mp4 https://media.signbsl.com/videos/asl/startasl/mp4/actor.mp4
!wget -O actor_2.mp4 https://www.handspeak.com/word/a/actor.mp4
!youtube-dl https://youtu.be/ZaxcUQkJ2a4 --output actor_3.mp4
!youtube-dl https://youtu.be/aYbsN9GG3kY --output actor_4.mp4


In [1]:
!wget https://zenodo.org/record/4010759/files/Adjectives_1of8.zip && unzip -q *.zip 

--2021-04-01 14:26:37--  https://zenodo.org/record/4010759/files/Adjectives_1of8.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1303983457 (1.2G) [application/octet-stream]
Saving to: ‘Adjectives_1of8.zip’

Adjectives_1of8.zip 100%[===================>]   1.21G  6.07MB/s    in 3m 28s  

2021-04-01 14:30:37 (5.97 MB/s) - ‘Adjectives_1of8.zip’ saved [1303983457/1303983457]



In [ ]:
%cd /content/
!pwd
%cd "Adjectives/3. happy"
!for i in *.MOV; do ffmpeg -y -loglevel error -i "$i" "${i%.*}.mp4"; done
!rm *.MOV
!zip -q -r happy.zip *.mp4 && mv happy.zip /content/ 
%cd ../..
!pwd